<a href="https://colab.research.google.com/github/kaggle-colab-comp/space-titanic/blob/main/DecisionTreeClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier


In [ ]:

training_data = "https://raw.githubusercontent.com/kaggle-colab-comp/space-titanic/main/train.csv?token=GHSAT0AAAAAACUPTFY62ALKTHUV77ZIAXVQZUKIL4Q"
train = pd.read_csv(training_data)
print(train.shape)
print(train.info())
train.head()

(8693, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
None


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [ ]:
test_data = 'https://raw.githubusercontent.com/kaggle-colab-comp/space-titanic/main/test.csv?token=GHSAT0AAAAAACUPTFY6H6MK4L3UHHUYADD6ZUKIOFQ'
test = pd.read_csv(test_data)
print(test.shape)
print(test.info())
test.head()


(4277, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB
None


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


Model


In [ ]:

# Drop rows with missing target values
train = train.dropna(subset=['Transported'])

# Select potentially less effective features
selected_features = ['VIP', 'CryoSleep']  # Categorical features that might not be very predictive

# Convert categorical features to integers
train['VIP'] = train['VIP'].astype(str).map({'False': 0, 'True': 1})
train['CryoSleep'] = train['CryoSleep'].astype(str).map({'False': 0, 'True': 1})
test['VIP'] = test['VIP'].astype(str).map({'False': 0, 'True': 1})
test['CryoSleep'] = test['CryoSleep'].astype(str).map({'False': 0, 'True': 1})

# Fill missing values
train[selected_features] = train[selected_features].fillna(0)  # Use a neutral value like 0
test[selected_features] = test[selected_features].fillna(0)

# Define features and target
X = train[selected_features]
y = train['Transported'].astype(int)  # Assuming 'Transported' is the target column and is binary

# Split the training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Decision Tree Classifier with very limited depth
clf = DecisionTreeClassifier(max_depth=1, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Validate the classifier
y_val_pred = clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy}")

# Prepare the test data for prediction
X_test = test[selected_features]  # Ensure the structure matches the training data

# Predict the test data
y_pred = clf.predict(X_test)


Validation Accuracy: 0.5048878665899943


In [ ]:
# Verify and fill any NaN values in 'VIP' and 'CryoSleep' columns
if 'VIP' in test.columns and 'CryoSleep' in test.columns:
    test['VIP'] = test['VIP'].fillna('False')
    test['CryoSleep'] = test['CryoSleep'].fillna('False')

    # Map categorical strings to integers with a fallback for unexpected NaNs
    mapping_dict = {'False': 0, 'True': 1, 'nan': 0}
    test['VIP'] = test['VIP'].astype(str).map(mapping_dict)
    test['CryoSleep'] = test['CryoSleep'].astype(str).map(mapping_dict)

    # Select only the features used in the model
    X_test = test[['VIP', 'CryoSleep']]
else:
    print("Required columns are missing in the test dataset.")

# Ensure there are no NaN values left; replace any that exist with 0 (assuming this aligns with your data's context)
X_test = X_test.fillna(0)

# Predict the test data
y_pred = clf.predict(X_test)

# Prepare the Kaggle submission file
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Transported': y_pred.astype(bool)  # Convert numerical predictions back to Boolean if necessary
})

# Write the DataFrame to a .csv file
submission.to_csv('first_attempt_submission_test.csv', index=False)




In [ ]:
# If using Google Colab, download the .csv file
from google.colab import files
files.download("first_attempt_submission_test.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>